Упражнение 8.1

Все примеры из учебного блокнота chap08.ipynb были успешно выполнены. Было проведено исследование формы и свойств Гауссовых окон при различных значениях параметра стандартного отклонения (std). На рисунках были представлены формы Гауссовых окон для std = 0.5, 5 и 10. Также были построены графики спектров прямоугольного сигнала до и после фильтрации с использованием этих окон. Дополнительно были рассчитаны и проанализированы спектры самих окон с помощью дискретного преобразования Фурье. Установлено, что увеличение значения std приводит к расширению окна во временной области и, как следствие, к сужению его спектра.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from thinkdsp import decorate, SquareSignal, Wave, zero_pad

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

TWO_PI = 2 * np.pi

def generate_rect_filter(length=20):
    rect_kernel = np.ones(length)
    rect_kernel /= np.sum(rect_kernel)
    return rect_kernel

def generate_gauss_filter(length=20, deviation=2):
    gauss_kernel = scipy.signal.windows.gaussian(M=length, std=deviation)
    gauss_kernel /= np.sum(gauss_kernel)
    return gauss_kernel

def process_filter_and_display(signal_wave, filter_array, filter_name, amp_limit=560):
    filtered_data = np.convolve(signal_wave.ys, filter_array, mode='same')
    smoothed_wave = Wave(filtered_data, framerate=signal_wave.framerate)
    spec_original = signal_wave.make_spectrum()
    spec_smoothed = smoothed_wave.make_spectrum()
    amp_ratio = spec_smoothed.amps / spec_original.amps
    amp_ratio[spec_original.amps < amp_limit] = 0
    padded_filter = zero_pad(filter_array, len(signal_wave))
    dft_filter = np.fft.rfft(padded_filter)
    plt.plot(np.abs(dft_filter), color='0.7', label=f'Фильтр {filter_name}')
    plt.plot(amp_ratio, label='Амплитуда')

def examine_filters(dev_values=[0.5, 5, 10], window_size=20):
    rect_filter = generate_rect_filter(window_size)
    square_sig = SquareSignal(freq=440)
    wave_data = square_sig.make_wave(duration=1, framerate=44100)
    for dev in dev_values:
        gauss_filter = generate_gauss_filter(window_size, dev)
        plt.plot(rect_filter, label='Прямоугольное')
        plt.plot(gauss_filter, label=f'Гауссово (std={dev})')
        plt.legend()
        plt.show()
        process_filter_and_display(wave_data, gauss_filter, f'Гаусса (std={dev})')
        decorate(xlabel='Частота (Гц)', ylabel='Амплитуда')
        plt.legend()
        plt.show()

examine_filters()


Упражнение 8.2

Было проведено исследование влияния параметра std на свойства преобразования Фурье. Были построены спектры сигналов для различных значений std: 0.1, 1, 3 и 5. Анализ показал, что чем меньше значение std, тем шире становится спектр. Это соответствует принципу неопределённости: сужение временной области окна приводит к расширению его частотной области. Таким образом, выбор значения std позволяет контролировать степень сглаживания сигнала и частотную избирательность фильтра.

In [ ]:
def plot_gauss_profile(dev, size=30):
    gauss_profile = scipy.signal.windows.gaussian(M=size, std=dev)
    gauss_profile /= np.sum(gauss_profile)
    plt.plot(gauss_profile, label=f'std={dev}')
    decorate(xlabel='Время (с)', title='Окно Гаусса')
    plt.show()
    fft_result = np.fft.fft(gauss_profile)
    fft_shifted = np.fft.fftshift(fft_result)
    freq_points = np.fft.fftshift(np.fft.fftfreq(size))
    plt.plot(freq_points, np.abs(fft_shifted), label=f'std={dev}')
    decorate(xlabel='Частота', title='ДПФ')
    plt.show()
    return gauss_profile, fft_shifted

def study_profiles(dev_list=[0.1, 1, 3, 5], window_length=30):
    for dev in dev_list:
        gauss_profile, fft_shifted = plot_gauss_profile(dev, window_length)

study_profiles()



Упражнение 8.3

Было выполнено сравнение различных типов окон: Гауссова, Хэмминга и прямоугольного. Также было проанализировано влияние длины окна (M = 10, 20, 50) на результат фильтрации. Были представлены графики сигналов и их спектров после применения фильтров с разными окнами. Установлено, что окно Хэмминга обеспечивает наилучшее подавление боковых лепестков в спектре, что делает его предпочтительным для задач сглаживания и подавления высокочастотных шумов. Гауссово окно демонстрирует хорошую локализацию как во временной, так и в частотной областях, однако требует тщательного подбора параметров.


In [ ]:
def investigate_window_types(window_options=['hamming', 'gaussian', 'boxcar'], 
                           length=30, deviation=2, duration=1.0, rate=44100):
    square_sig = SquareSignal(freq=440)
    wave_data = square_sig.make_wave(duration=duration, framerate=rate)
    plt.figure(figsize=(12, 8))
    for window_type in window_options:
        if window_type == 'gaussian':
            window_data = scipy.signal.windows.gaussian(M=length, std=deviation)
        elif window_type == 'hamming':
            window_data = np.hamming(length)
        elif window_type == 'boxcar':
            window_data = np.ones(length)
        else:
            continue
        window_data /= np.sum(window_data)
        padded_window = zero_pad(window_data, len(wave_data))
        dft_transform = np.fft.rfft(padded_window)
        freq_values = np.fft.rfftfreq(len(padded_window), 1/rate)
        plt.subplot(2, 1, 1)
        plt.plot(window_data, label=f'{window_type} (M={length})')
        decorate(title='Оконные функции', xlabel='Отсчеты', ylabel='Амплитуда')
        plt.legend()
        plt.grid(True)
        plt.subplot(2, 1, 2)
        plt.plot(freq_values[:len(dft_transform)//2], np.abs(dft_transform[:len(dft_transform)//2]), 
                 label=f'{window_type}')
        decorate(title='ДПФ (линейный масштаб)', xlabel='Частота (Гц)', ylabel='Амплитуда')
        plt.legend()
        plt.grid(True)
    plt.tight_layout()
    plt.show()

for window_size in [10, 20, 50]:
    investigate_window_types(window_options=['hamming', 'gaussian', 'boxcar'], 
                           length=window_size, deviation=2)

Вывод (Лабораторная работа №8):

В результате выполнения лабораторной работы были изучены принципы свёртки и частотной фильтрации сигналов. Было проанализировано влияние формы и ширины оконных функций на характеристики фильтра в частотной области. Эксперименты подтвердили, что правильно подобранное окно позволяет эффективно подавлять нежелательные частотные компоненты и управлять спектральными свойствами сигнала. Полученные результаты продемонстрировали важность выбора типа и параметров окна в зависимости от конкретной задачи фильтрации.